In [1]:
import zipfile
import os
import tempfile
import geopandas as gpd

def zip_gpkg_to_geojson(zip_pfad, geojson_pfad, layername=None):
    # Temporäres Verzeichnis zum Entpacken
    with tempfile.TemporaryDirectory() as tmpdir:
        # ZIP entpacken
        with zipfile.ZipFile(zip_pfad, 'r') as zip_ref:
            zip_ref.extractall(tmpdir)

        # Suche nach der GPKG-Datei im temporären Ordner
        gpkg_datei = None
        for root, dirs, files in os.walk(tmpdir):
            for file in files:
                if file.lower().endswith(".gpkg"):
                    gpkg_datei = os.path.join(root, file)
                    break
            if gpkg_datei:
                break

        if not gpkg_datei:
            raise FileNotFoundError("Keine .gpkg-Datei im ZIP gefunden!")

        print(f"GPKG gefunden: {gpkg_datei}")

        # GPKG laden (Layer optional angeben)
        gdf = gpd.read_file(gpkg_datei, layer=layername)
        print(f"Layer geladen mit CRS: {gdf.crs}")

        # Falls CRS nicht gesetzt oder falsch: explizit setzen
        if gdf.crs is None:
            gdf.set_crs(epsg=2056, inplace=True)
            print("CRS auf EPSG:2056 gesetzt")

        # Transformieren zu WGS84
        gdf_wgs84 = gdf.to_crs(epsg=4326)

        # Speichern als GeoJSON
        gdf_wgs84.to_file(geojson_pfad, driver="GeoJSON")
        print(f"GeoJSON gespeichert unter: {geojson_pfad}")

# Beispielnutzung:
zip_datei = "data/swissboundaries3d_2025-04_2056_5728.gpkg.zip"
ausgabe_geojson = "swissBOUNDARIES3D_1_3_TLM_GEMEINDEGRENZEN.geojson"

zip_gpkg_to_geojson(zip_datei, ausgabe_geojson)


GPKG gefunden: C:\Users\sarah\AppData\Local\Temp\tmp18f10kcw\swissBOUNDARIES3D_1_5_LV95_LN02.gpkg


c:\Users\sarah\anaconda3\envs\4040_geopython310\lib\site-packages\pyogrio\geopandas.py:265: UserWarning: More than one layer found in 'swissBOUNDARIES3D_1_5_LV95_LN02.gpkg': 'tlm_hoheitsgrenze' (default), 'tlm_hoheitsgebiet', 'tlm_bezirksgebiet', 'tlm_landesgebiet', 'tlm_kantonsgebiet'. Specify layer parameter to avoid this warning.
  result = read_func(


Layer geladen mit CRS: EPSG:2056
GeoJSON gespeichert unter: swissBOUNDARIES3D_1_3_TLM_GEMEINDEGRENZEN.geojson
